In [ ]:
## Foot Traffic Analysis Using Object Detection and Tracking

#The goal of this project was to perform foot traffic analysis by counting how
#many people enter a shop using computer vision techniques.

#For this project, a CCTV video from a passenger counter in a bus was obtained
#from Kaggle.com. The focus of the analysis was the bus door, where
#foot traffic was monitored as passengers got on board.

#To achieve this, the following tools were used:
#**OpenCV**: For video processing.
#**YOLOv8**: For object detection.
#**Ultralytics**: For tracking and integrating the YOLO model.

#In this notebook, we will demonstrate the entire process, from video loading
#and detection to foot traffic tracking.


In [ ]:
# Install required packages
!pip install opencv-python ultralytics cvzone

In [ ]:
### Saving Images from Video

#In this step, we will capture 200 images from the provided video (`fdoor.avi`)
#for later use in the object detection and tracking process.

#We will resize the frames to 1080x500 pixels, display each frame in a window,
#and save them to a directory.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Download fdoor.avi Video from Google Drive
# Install gdown
!pip install gdown

# Use gdown to download the file
!gdown https://drive.google.com/uc?id=1hJ8RcS3yVwdJtPeIFRQtZBOqa-vWsfBH

# Check if the file has been downloaded
!ls -l


Downloading...
From: https://drive.google.com/uc?id=1hJ8RcS3yVwdJtPeIFRQtZBOqa-vWsfBH
To: /content/fdoor.avi
100% 10.2M/10.2M [00:00<00:00, 19.3MB/s]
total 9948
drwx------ 5 root root     4096 Sep  7 08:48 drive
-rw-r--r-- 1 root root 10176568 Sep  7 08:48 fdoor.avi
drwxr-xr-x 1 root root     4096 Sep  5 17:47 sample_data


In [ ]:
import cv2

# Path to the downloaded video file
video_path = 'fdoor.avi'

# Initialize video capture using OpenCV
cap = cv2.VideoCapture(video_path)

# Check if the video file is opened correctly
if not cap.isOpened():
    print("Error: Could not open video file.")
else:
    print("Video file loaded successfully!")


Video file loaded successfully!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import cv2
import time

# Initialize counters and video capture
cpt = 0
maxFrames = 200  # Set the maximum number of frames to capture
cap = cv2.VideoCapture('fdoor.avi')  # Load the video

# Define the directory path in Google Drive
save_directory = '/content/drive/My Drive/images/'

# Loop through the video and save images
while cpt < maxFrames:
    ret, frame = cap.read()  # Read each frame
    if not ret:
        break
    frame = cv2.resize(frame, (1080, 500))  # Resize the frame

    # Save the frame as an image
    cv2.imwrite(f"{save_directory}person_{cpt}.jpg", frame)

    time.sleep(0.01)  # Add a slight delay to mimic real-time capture
    cpt += 1

# Release video capture and close all windows
cap.release()
cv2.destroyAllWindows()

print("All images have been successfully saved to Google Drive.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
All images have been successfully saved to Google Drive.


In [ ]:
### Install and Launch LabelImg

#Next, we will use **LabelImg** to annotate the images. The objective is to
#create a "person" class for the passengers entering the door and label
#their heads and faces as they appear in the video frames.

#Steps:
#1. Install **LabelImg**.
#2. Launch it and create the "person" class.
#3. Skip all frames where the door is not opened.
#4. Draw a rectangle around the head and face of each passenger as they
#enter the door, but not fully inside the bus.
#5. Save the annotations to the same directory as the images.

#The save directory is: '/content/drive/My Drive/images/'
#After annotating, we will clean up extra images that do not contain
#passengers, using the code in the next step.


In [ ]:
# Define the image directory path
import os

# Path to your images directory
images_dir = '/content/drive/MyDrive/images'

# List all image files in the directory
image_files = os.listdir(images_dir)
print(image_files)

# Get a list of image files in the directory
image_files = [f for f in os.listdir(images_dir) if os.path.isfile(os.path.join(images_dir, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

# Iterate through the image files
for image_file in image_files:
    # Extract the image name without extension
    image_name = os.path.splitext(image_file)[0]
    # Look for a corresponding .xml annotation file
    txt_file = image_name + ".txt"

    # If the .txt file doesn't exist, delete the image
    if not os.path.exists(os.path.join(images_dir, txt_file)):
        os.remove(os.path.join(images_dir, image_file))
        print(f"Deleted {image_file} because {txt_file} does not exist.")


['person_104.txt', 'person_100.txt', 'person_51.txt', 'person_68.txt', 'person_74.txt', 'person_59.txt', 'person_88.txt', 'person_49.txt', 'person_130.txt', 'person_129.txt', 'person_79.txt', 'person_75.txt', 'person_71.txt', 'person_73.txt', 'person_50.txt', 'person_56.txt', 'person_69.txt', 'person_48.txt', 'person_131.txt', 'person_121.txt', 'person_124.txt', 'person_83.txt', 'person_67.txt', 'person_76.txt', 'person_123.txt', 'person_122.txt', 'person_87.txt', 'person_60.txt', 'person_117.txt', 'person_70.txt', 'person_102.txt', 'person_132.txt', 'person_115.txt', 'person_118.txt', 'person_128.txt', 'person_113.txt', 'person_109.txt', 'person_78.txt', 'person_57.txt', 'person_82.txt', 'person_54.txt', 'person_65.txt', 'person_116.txt', 'person_119.txt', 'person_77.txt', 'person_86.txt', 'person_105.txt', 'person_80.txt', 'person_72.txt', 'person_112.txt', 'person_84.txt', 'person_114.txt', 'person_125.txt', 'person_85.txt', 'person_52.txt', 'person_126.txt', 'person_55.txt', 'perso

In [ ]:
### Organize and Prepare Data

#To prepare the data for training and validation, we will organize
#the images and annotations into the following folder structure:

# [**foottraffic/**]
#   (**images/**){**validation/**}, { **training/**}
#  (**labels/**){**validation/**}, { **training/**}


#Copy the images and their corresponding `.txt` annotation files into
#the appropriate validation and training folders. After organizing the
#files, we will zip the `foottraffic` folder for uploading.

#The next cell will handle the folder creation and file copying.


In [ ]:
import os
import shutil

# Define the root directory for the new folder structure
root_dir = 'foottraffic1'
image_dir = os.path.join(root_dir, 'images')
label_dir = os.path.join(root_dir, 'labels')

# Create the directory structure
for folder in [image_dir, label_dir]:
    os.makedirs(os.path.join(folder, 'validation'), exist_ok=True)
    os.makedirs(os.path.join(folder, 'training'), exist_ok=True)

# Define the original images and labels directory (same directory in your case)
original_dir = '/content/drive/MyDrive/images'

# Copy all images and labels to both training and validation folders
def copy_files(src_dir, dest_image_dir, dest_label_dir):
    for file_name in os.listdir(src_dir):
        src_file = os.path.join(src_dir, file_name)
        if os.path.isfile(src_file):
            # Copy the file to both training and validation subfolders
            for subfolder in ['training', 'validation']:
                if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    shutil.copy(src_file, os.path.join(dest_image_dir, subfolder, file_name))
                elif file_name.lower().endswith('.txt'):
                    shutil.copy(src_file, os.path.join(dest_label_dir, subfolder, file_name))

# Copy images and labels into the new folder structure
copy_files(original_dir, image_dir, label_dir)

# Zip the foottraffic1 folder
shutil.make_archive('foottraffic1', 'zip', root_dir)

print("Zipping complete: foottraffic1.zip created.")


Zipping complete: foottraffic1.zip created.


In [ ]:
### Upload the ZIP File to Google Drive

#The zip file of the `foottraffic` folder has been created. You can upload this
#file to Google Drive for easy sharing and access.

# Move the zip file to Google Drive
shutil.move('foottraffic1.zip', '/content/drive/MyDrive/foottraffic1.zip')

print("Zipping complete and file uploaded to Google Drive: foottraffic1.zip.")

Zipping complete and file uploaded to Google Drive: foottraffic1.zip.


In [ ]:
### Access the Essential Colab Notebook

#You can access the Colab notebook that contains the YOLOv8 model code and
#configurations necessary for this project. Feel free to run, modify, and adapt
#the notebook as needed.

from IPython.display import display, HTML

# Display the link as HTML in a code cell
display(HTML('<a href="https://colab.research.google.com/drive/1WljA5ADBqSd_YeOhw4iPvN1tb6fAgd8r?usp=sharing" target="_blank">Open the Colab Notebook</a>'))



In [ ]:
### Create `data.yaml` File and Train the Model

#After mounting Google Drive in the Colab notebook, we need to create a `data.yaml` 
#file in the `foottraffic` folder. This file will contain the configuration for 
#training, including paths to training and validation data, number of classes, and class names.

#We will then proceed to run the YOLOv8 training process using the provided Colab notebook.

#The `data.yaml` file will have the following content:
# Create the data.yaml file
data_yaml_content = """\
train: ../foottraffic/images/training
val: ../foottraffic/images/validation
# number of classes
nc: 1
# class names
names: ['person']
"""

# Define the path for the data.yaml file
yaml_file_path = '/content/data.yaml'

# Write the content to the file
with open(yaml_file_path, 'w') as file:
    file.write(data_yaml_content)

print("data.yaml file created successfully!")

# Copy the data.yaml file to Google Drive
shutil.copy(yaml_file_path, '/content/drive/MyDrive/data.yaml')

print("data.yaml file successfully uploaded to Google Drive!")


data.yaml file created successfully!
data.yaml file successfully uploaded to Google Drive!


In [ ]:
### Run YOLOv8 Training and Download the Best Model

#From the YOLOv8 training using the Colab notebook, locate the `best.pt` model
#in the `runs/detect/train` directory and download it.

#Once the model training is complete, download the `best.pt` file to use for
#object detection.

from google.colab import files
from google.colab import drive

# Step 1: Manually upload the file from your local machine
uploaded = files.upload()

# The file is now available in the Colab environment under /content/
# Check uploaded files
print("Uploaded files:", os.listdir('/content'))

# Define the path to the uploaded 'best.pt' file
uploaded_file_path = '/content/best.pt'  # Adjust the name if different

# Step 2: Mount Google Drive
drive.mount('/content/drive')

# Define the destination path in your Google Drive
destination_path = '/content/drive/MyDrive/best.pt'  # Modify this path if needed

# Move the file to Google Drive
shutil.move(uploaded_file_path, destination_path)

print(f"best.pt file successfully uploaded to Google Drive!")



Saving best.pt to best.pt
Uploaded files: ['.config', 'best.pt', 'coco1.txt', 'foottraffic1', 'foottraffic1.zip', 'fdoor.avi', 'drive', 'data.yaml', 'sample_data']
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
best.pt file successfully uploaded to Google Drive!


In [ ]:
# Define the content for coco1.txt
coco_txt_content = "person"  # The class used for tracking (passengers)

# Define the path where the coco1.txt file will be created locally
coco_txt_path = '/content/coco1.txt'

# Write the content ("person") to the coco1.txt file
with open(coco_txt_path, 'w') as file:
    file.write(coco_txt_content)

print("coco1.txt file created successfully with content 'person'!")

# Define the destination folder in Google Drive
destination_path = '/content/drive/MyDrive/coco1.txt'  # Modify the path if needed

# Move the coco1.txt file to Google Drive
shutil.move(coco_txt_path, destination_path)

print(f"coco1.txt file successfully uploaded to Google Drive!")

coco1.txt file created successfully with content 'person'!
coco1.txt file successfully uploaded to Google Drive!


In [ ]:
#Analyze Video with YOLOv8 Model
### Foot Traffic Detection with `test.py`

#The `test.py` script integrates YOLOv8 for object detection and tracking. 
#It detects passengers entering the bus through a designated zone and counts 
#them using unique IDs.

#This script performs the following:
#1. Loads the trained YOLOv8 model (`best.pt`).
#2. Tracks objects and detects when they cross a defined zone.
#3. Counts the number of passengers entering the bus.


In [ ]:
## Analyze the Video

#Click the link below to download the `test.py` code. After downloading, 
#open the file using your preferred IDE or platform:

#[Download test.py]
(https://drive.google.com/uc?id=1kyNTZF5nGq5-1ENNECAlgvoaWxDbNZT1&export=download)

### Instructions

#1. Download the file from the link above.
#2. Open the `test.py` file using your preferred IDE preferably Thonny.
#3. Open the video file in the same IDE or a video player to analyze it.

#Feel free to reach out if you have any questions or need further assistance. KihoroBrian.


SyntaxError: invalid decimal literal (<ipython-input-5-899c3d967ad7>, line 5)

In [12]:
from google.colab import files
uploaded = files.upload()

Saving foottraffic_video.avi to foottraffic_video.avi


In [13]:
from IPython.display import HTML

video_path = 'foottraffic_video.avi'

def display_video(video_path):
    video_tag = f'''
    <video width="1020" height="500" controls>
        <source src="{video_path}" type="video/mp4">
        Your browser does not support the video tag.
    </video>
    '''
    return HTML(video_tag)

# Display the video
display_video(video_path)
